<a href="https://colab.research.google.com/github/preetsanjam/Crypto_Price_Analysis/blob/main/Sentiment_Dogecoin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import mwclient
import time
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')

site = mwclient.Site('en.wikipedia.org')
page = site.pages['Dogecoin']

In [ ]:
revs = list(page.revisions())

In [ ]:
revs[0]

OrderedDict([('revid', 1260257932),
             ('parentid', 1257491874),
             ('user', 'FifthFive'),
             ('timestamp',
              time.struct_time(tm_year=2024, tm_mon=11, tm_mday=29, tm_hour=19, tm_min=36, tm_sec=15, tm_wday=4, tm_yday=334, tm_isdst=-1)),
             ('comment',
              '/* Currency supply */ fix typo, implementing edit request by 207.96.32.81 on article talk')])

In [ ]:
revs = sorted(revs, key=lambda rev:rev['timestamp'])

In [ ]:
revs[0]

OrderedDict([('revid', 585981768),
             ('parentid', 0),
             ('user', 'Citation Needed'),
             ('timestamp',
              time.struct_time(tm_year=2013, tm_mon=12, tm_mday=14, tm_hour=0, tm_min=13, tm_sec=28, tm_wday=5, tm_yday=348, tm_isdst=-1)),
             ('comment',
              'Yes, this is going to be an article on a cryptocurrency based on a internet meme.')])

In [ ]:
from transformers import pipeline
sentiment_pipleine = pipeline('sentiment-analysis', model='distilbert/distilbert-base-uncased-finetuned-sst-2-english', revision='714eb0f')

def find_sentiment(text):
  sent = sentiment_pipleine(text[:250])[0]
  score = sent['score']
  if sent['label'] == 'NEGATIVE':
    score *= -1
  return score

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
edits = {}

for rev in revs:
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)

    edits[date]["edit_count"] += 1

    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))

In [ ]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0

    del edits[key]["sentiments"]

In [ ]:
import pandas as pd

edits_df = pd.DataFrame.from_dict(edits, orient="index")

In [ ]:
edits_df

,edit_count,sentiment,neg_sentiment
2013-12-14,110,0.480359,0.181818
2013-12-15,47,-0.253898,0.595745
2013-12-16,16,0.031845,0.437500
2013-12-17,1,-0.998040,1.000000
2013-12-18,1,0.748121,0.000000
...,...,...,...
2024-11-03,2,-0.125821,0.500000
2024-11-06,1,0.597184,0.000000
2024-11-14,1,-0.999224,1.000000
2024-11-15,1,0.930021,0.000000


In [ ]:
edits_df.index = pd.to_datetime(edits_df.index)

In [ ]:
from datetime import datetime

dates = pd.date_range(start="2013-12-14",end=datetime.today())

In [ ]:
edits_df = edits_df.reindex(dates, fill_value=0)

In [ ]:
edits_df

,edit_count,sentiment,neg_sentiment
2013-12-14,110,0.480359,0.181818
2013-12-15,47,-0.253898,0.595745
2013-12-16,16,0.031845,0.437500
2013-12-17,1,-0.998040,1.000000
2013-12-18,1,0.748121,0.000000
...,...,...,...
2024-11-26,0,0.000000,0.000000
2024-11-27,0,0.000000,0.000000
2024-11-28,0,0.000000,0.000000
2024-11-29,1,-0.995401,1.000000


In [ ]:
rolling_edits = edits_df.rolling(30, min_periods=30).mean()

In [ ]:
rolling_edits = rolling_edits.dropna()

In [ ]:
rolling_edits

,edit_count,sentiment,neg_sentiment
2014-01-12,8.766667,-0.113297,0.424761
2014-01-13,5.133333,-0.161805,0.452034
2014-01-14,3.666667,-0.145299,0.443287
2014-01-15,3.133333,-0.146360,0.428704
2014-01-16,3.133333,-0.146335,0.428704
...,...,...,...
2024-11-26,0.166667,0.013405,0.050000
2024-11-27,0.166667,0.013405,0.050000
2024-11-28,0.166667,0.013405,0.050000
2024-11-29,0.200000,-0.019775,0.083333


In [ ]:
rolling_edits.to_csv("doge_wikipedia_edits.csv")